# Поиск токсичных комментариев

__Описание проекта__:   
Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.  

__Цель проекта__:  
Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.
Постройте модель со значением метрики качества F1 не меньше 0.75. 

__Ход исследования__:  
Шаг 1. Загрузка данных  
Шаг 2. Изучение данных    
Шаг 3. Лемматизация  
Шаг 4. Подготовка данных к обучению  
Шаг 5. Обучение моделей   
Шаг 6. Выводы




In [ ]:
#!pip install spacy
#!python -m spacy download en_core_web_sm

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
import spacy
import matplotlib.pyplot as plt
from tqdm import tqdm
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV

## Шаг 1. Загрузка данных

In [ ]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv', sep=',')
data.head()

## Шаг 2. Изучение данных

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data.duplicated().sum()

Дубликатов в нашем датафрейме нет

In [ ]:
data.isna().sum()

Как нет и пропусков

In [ ]:
data['toxic'].value_counts(normalize=True)

In [ ]:
data['toxic'].value_counts().plot(kind = 'pie', autopct = '%.2f')
plt.title('Отношение позитивных и токсичных комментариев')

Присутсвует дисбаланс классов

## Шаг 3. Лемматизация 

In [ ]:
nltk.download('stopwords') 
nltk.download('punkt')
nltk.download('wordnet')

In [ ]:
def lemmatize(text, lemmatizer):
    doc = lemmatizer(text)
    lemm_text = " ".join([token.lemma_ for token in doc])
        
    return lemm_text

In [ ]:
def clear(text):
    cleaned = re.sub(r"[^a-zA-Z\' ]", ' ', text)
    return " ".join(cleaned.split())

In [ ]:
sp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])


In [ ]:
data['predproc'] = data['text'].apply(lemmatize, lemmatizer=sp)

In [ ]:
data.head()

In [ ]:
data['predproc'] = data['predproc'].apply(clear)

In [ ]:
data.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
data.head()

Данные очищены от лишних символов, лемматизированы, векторизированы

## Шаг 4.Подготовка данных к обучению

In [ ]:
features = data['predproc']
target = data['toxic']

X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.25, random_state=88)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=88)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
print(X_val.shape, y_val.shape)

In [ ]:
stop_words = set(stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stop_words) 

In [ ]:
X_train_vector = count_tf_idf.fit_transform(X_train)
X_test_vector = count_tf_idf.transform(X_test)
X_val_vector = count_tf_idf.transform(X_val)

Созданы тренировочная, валидационная и тестовая выборки

## Шаг 5. Обучение моделей

Логистическая регрессия

In [ ]:
model_logistic = LogisticRegression()

param_grid_logistic = {
    'penalty': ['l2', 'none'],
    'C': [5, 10, 15],
}

gs_logistic = GridSearchCV(
    model_logistic,
    param_grid=param_grid_logistic,
    scoring='f1',
    n_jobs=-1
)

gs_logistic.fit(X_train_vector, y_train)
best_params_logistic = gs_logistic.best_params_
print(f'best_score: {gs_logistic.best_score_}')

print(f'best_params: {gs_logistic.best_params_}')

In [ ]:
best_model_lr =  LogisticRegression(**best_params_logistic)
best_model_lr.fit(X_train_vector, y_train)

In [ ]:
val_predictions_lr = best_model_lr.predict(X_val_vector)

In [ ]:
f1_lr = f1_score(y_val, val_predictions_lr)
f1_lr

модель DecisionTreeClassifier

In [ ]:
model_decision_tree = DecisionTreeClassifier()

param_grid_decision_tree = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [1, 10, 20, 30],
}

gs_decision_tree = GridSearchCV(
    model_decision_tree,
    param_grid=param_grid_decision_tree,
    scoring='f1',
    n_jobs=-1
)

gs_decision_tree.fit(X_train_vector, y_train)
best_params_decision_tree = gs_decision_tree.best_params_
print(f'best_score: {gs_decision_tree.best_score_}')

print(f'best_params: {gs_decision_tree.best_params_}')

In [ ]:
best_model_dt =  DecisionTreeClassifier(**best_params_decision_tree)
best_model_dt.fit(X_train_vector, y_train)

In [ ]:
val_predictions_dt = best_model_dt.predict(X_val_vector)

In [ ]:
f1_dt = f1_score(y_val, val_predictions_dt)
f1_dt

модель CatBoostClassifier

In [ ]:
model_catboost = CatBoostClassifier(auto_class_weights='Balanced')

param_grid_catboost = {
    'iterations': [10],
    'learning_rate': [0.01]
}

gs_catboost = GridSearchCV(
    model_catboost,
    param_grid=param_grid_catboost,
    scoring='f1',
    n_jobs=-1
)

gs_catboost.fit(X_train_vector, y_train)
best_params_catboost = gs_catboost.best_params_

print(f'best_params: {gs_catboost.best_params_}')

In [ ]:
best_model_catboost =  CatBoostClassifier(**best_params_catboost)
best_model_catboost.fit(X_train_vector, y_train)

In [ ]:
val_predictions_catboost = best_model_catboost.predict(X_val_vector)

In [ ]:
f1_catboost = f1_score(y_val, val_predictions_catboost)
f1_catboost

Обучены три модели и для каждой подсчитано значение F1 меры на валидационной выборке

## Шаг 6 Выводы

Модель LogisticRegression показала лучший результат на валидационной выборке, проверим показатель F1 меры этой модели на тестовой выборке 

In [ ]:
final_predictions = best_model_lr.predict(X_test_vector)
f1_final = f1_score(y_test, final_predictions)
f1_final

Модель LogisticRegression показала значение метрики F1 на тестовой выборке равное 0.7757197872730606, что проходит порог в 0.75

# Общий Вывод

В этом проекте мы загрузили данные и провели их предобработку, обучили 3 модели с разлчиными гиперпараметрами и подсчитали для них значение метрики F1 на валидационной выборке.
Лучшей оказалась LogisticRegression, для нее мы и подсчитали значение метрики F1 для тестовой выборки, получили результат в 0.7757197872730606, что больше 0.75 и удовлетворяет условию.